In [ ]:
#Load python modules : Plot, image , numerical python 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from scipy.interpolate import interp1d

# Set a useful font library
font1 = {'size'   : 20, 'family':'STIXGeneral'}
#Colors for plotting 
colors = ['teal', 'yellowgreen', 'gold']
#line width for plottting
lw = 2

# We will start with a regression problem using sklearn 

In [ ]:
#Load the power spectrum data. Columns: k , P(k) w BAO feature , P(k) w/o BAO feature
dat1 = np.loadtxt("data/PT_z0.dat")
length = len(dat1);

#set training data proportion 
m = int(length*0.5)

k_pt = dat1[:,0]
baotab = dat1[:,1]/dat1[:,2]
bao =  interp1d(k_pt, baotab, kind='linear')

In [ ]:
# Plot BAO signal

# Set figure size 
plt.figure(figsize=(5,5))
ax = plt.subplot(1,1,1)

#Plot 1st column against 2nd column 
plt.plot(k_pt, bao(k_pt), label='$z=0$', alpha=0.6 , linestyle='solid', marker='x', color='red')

#Plot the legend
plt.legend(loc='best',prop={'size': 16})

#Set ranges
plt.ylim(0.9,1.1)
plt.xlim(0.001,0.5)

#Format axes 
ax.tick_params(axis='both', which='major', labelsize=15)
plt.ylabel('$ P(k)/P_{NW}(k)$',**font1)
plt.xlabel('$k [h/{\\rm Mpc}]$', **font1)
plt.legend(loc='best',prop={'size': 16})

In [ ]:
# Let's try to apply linear regression to this problem....

#Load sklearn regression algorithm 
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

# Set degree of polynomial, regularisation and maximum k to plot
mylam = 0
degree = 1
kmax = 0.2

# Training set 
train_k = dat1[:m,0]
train_bao = bao(train_k)

# Test set 
test_k = dat1[:,0]
test_bao = bao(test_k)


# create matrix versions of these arrays
X = train_k[:, np.newaxis]
X_plot = test_k[:, np.newaxis]


# The ridge model fits with a cost function : 
#  J(w) = ||y - Xw||^2_2 + alpha * ||w||^2_2  

mycost = [(Ridge(alpha=mylam))]

model = make_pipeline(PolynomialFeatures(degree), estimator)
model.fit(X, train_bao)
y_plot = model.predict(X_plot)

plt.figure(figsize=(12,5))
ax = plt.subplot(1,2,1)

plt.plot(test_k, test_bao, color='cornflowerblue', linewidth=lw,label="Truth")

plt.scatter(train_k, train_bao, color='navy', s=30, marker='o', label="training points")

plt.plot(test_k, y_plot, color=colors[count], linewidth=lw, label="degree: %d" % degree)


plt.legend(loc='lower left')
#Set ranges
plt.ylim(0.9,1.1)
plt.xlim(0.001,kmax)

#Format axes 
ax.tick_params(axis='both', which='major', labelsize=15)
plt.ylabel(' BAO signal ',**font1)
plt.xlabel('$k [h/{\\rm Mpc}]$', **font1)
plt.legend(loc='best',prop={'size': 16})

ax = plt.subplot(1,2,2)
if (degree>20 and kmax<0.26): 
    img = mpimg.imread('data/bor2.jpg')
else : 
    img = mpimg.imread('data/bor1.jpg')
    
plt.imshow(img)

ax.set_xticks([])
ax.set_yticks([])

# Let's try a classification example: power spectra into f(R) and GR  classes

In [ ]:
# Load some examples to investigate 

exampleno = 50

#Columns: k , P(k) w BAO feature , P(k) w/o BAO feature
fr_exp = np.loadtxt('data/class/fr_mlg/'+str(exampleno)+'.txt')
gr_exp = np.loadtxt('data/class/gr_mlg/'+str(exampleno)+'.txt')


plt.figure(figsize=(12,5))

ax = plt.subplot(1,2,1)
#Pure spectra 
plt.plot(fr_exp[:,0],fr_exp[:,1], color='r')
plt.plot(gr_exp[:,0],gr_exp[:,1], color='b')

ax.tick_params(axis='both', which='major', labelsize=15)
plt.ylabel(' $P(k)$ ',**font1)
plt.xlabel('$k [h/{\\rm Mpc}]$', **font1)

ax = plt.subplot(1,2,2)
#BAO signal 
plt.plot(fr_exp[:,0],fr_exp[:,1]/fr_exp[:,2], color='r', label='f(R)')
plt.plot(gr_exp[:,0],gr_exp[:,1]/gr_exp[:,2], color='b', label='GR')

ax.tick_params(axis='both', which='major', labelsize=15)
plt.ylabel(' BAO signal ',**font1)
plt.xlabel('$k [h/{\\rm Mpc}]$', **font1)
plt.legend(loc='best',prop={'size': 16})


There is a clear enhancement of power in f(R) cosmologies - we can use this to classify the spectra 

We will load the values very crudely - Pandas tutorial will teach us how to do this more elegantly! 

In [ ]:
# Load a high k value array from k = 0.1 to 0.25
dat2 = np.loadtxt('data/class/gr_mlg/1.txt')
kvalues = fr_exp[20:50,0]
print(kvalues)

In [ ]:
# Get number of examples per class 
import os
path, dirs, files = next(os.walk("./data/class/fr_mlg"))
file_count = len(files) # We set aside 100 values for testing!  

# number of files to be included in test set 
test_set = 101

# Define an array to store the various values of P(k) at a low k and another at a high k
high_fr = []
high_gr = []
low_fr = []
low_gr = []


#Maximum and minimum index to find maximum and minimum of spectrum 
maxi = 50
mini = 20

# Load the P(k) values per k 
for i in range(file_count-test_set):
    datfr = np.loadtxt('data/class/fr_mlg/'+str(i+1)+'.txt')
    datgr = np.loadtxt('data/class/gr_mlg/'+str(i+1)+'.txt')
    
    #perform rough feature normalisation 
    norm = np.mean(datgr[mini:maxi,1])
    
    high_fr.append(np.max(datfr[mini:maxi,1]/norm))
    low_fr.append(np.min(datfr[mini:maxi,1]/norm))  
    high_gr.append(np.max(datgr[mini:maxi,1]/norm))
    low_gr.append(np.min(datgr[mini:maxi,1]/norm))

    
    
total_low = np.concatenate((low_gr,low_fr), axis=None)
total_high = np.concatenate((high_gr,high_fr), axis=None)


plt.plot(low_fr,high_fr, linestyle='none',marker='x', label='f(R)')
plt.plot(low_gr,high_gr, linestyle='none',marker='o', label='GR')
plt.ylabel('Maximum $P(k)$ ',**font1)
plt.xlabel('Minimum $P(k)$', **font1)
plt.legend(loc='best',prop={'size': 16})


In [ ]:
# Let's now fit a decision boundary to this - linear will work well

# Define logistic function first 
def sigmoid(x):
    return 1/(1+np.exp(-x))

xplot = np.linspace(-5,5,100)

plt.plot(xplot,sigmoid(xplot))
plt.axvline(x=0., ymin=0., ymax=10,color='k')
plt.axhline(y=1., xmin=-5, xmax=5,color='k',linestyle='dotted')
plt.axhline(y=0., xmin=-5, xmax=5,color='k',linestyle='dotted')

In [ ]:
#Define our cost function 
# x is an 3 x m matrix - (1, x1, x2) per example , x1 and x2 are our features (max and min P(k))
# y is an m x 1 matrix - 0 or 1 per example 
# w is an 3 x 1 matrix - (bias, w1, w2) 
# w.T is w transpose 

def mycost(w,x,y,m):
    myunity = np.ones(m)
    sigmoidmat = sigmoid(np.matmul(x.T,w))    
    term1 = np.matmul(y.T,np.log(sigmoidmat))
    
    term2 = np.matmul((myunity-y).T,np.log(myunity-sigmoidmat))
    
    return -1/m * (term1 + term2)

In [ ]:
# Create matrix of features 
bias = np.ones((file_count-test_set)*2)
training_x = np.concatenate(([bias],[total_low],[total_high]))
# Create vector of 'right' answers 
training_y = np.concatenate((np.zeros((file_count-test_set)),np.ones((file_count-test_set))))

# Test cost function for initial guess 
winit = np.array([10,-10,1])
mycost(winit, training_x, training_y , (file_count-test_set)*2)


In [ ]:
# Use scipy module to optimise the function using a gradient descent algorithm
import scipy.optimize as optimize

xopt = optimize.fmin(mycost, winit , args=(training_x, training_y , (file_count-test_set)*2), xtol=1e-5, disp=True)

print(xopt)

In [ ]:
#Plot decision boundary
def decbound(w,x):
    return -(w[0] + w[1]*x)/w[2]


plt.figure(figsize=(12,5))
ax = plt.subplot(1,2,1)
plt.plot(total_low, decbound(xopt,total_low))
plt.plot(low_fr,high_fr, linestyle='none',marker='x', label='f(R)')
plt.plot(low_gr,high_gr, linestyle='none',marker='o', label='GR')
plt.ylabel('Maximum $P(k)$ ',**font1)
plt.xlabel('Minimum $P(k)$', **font1)
plt.legend(loc='best',prop={'size': 16})


ax = plt.subplot(1,2,2)
img = mpimg.imread('data/bor3.jpg')
plt.imshow(img)

ax.set_xticks([])
ax.set_yticks([])

In [ ]:
# Let's check how accurate our fit is 

# Define a prediction based on our fit 
def predictor(x1,x2):
    if(sigmoid(xopt[0] + xopt[1]*x1 + xopt[2]*x2)>=0.5):
        return 1
    return 0

# Create test data from the  unused examples
high_frt = []
high_grt = []
low_frt = []
low_grt = []

for i in range(file_count-test_set,file_count-1):
    
    datfr = np.loadtxt('data/class/fr_mlg/'+str(i+1)+'.txt')
    datgr = np.loadtxt('data/class/gr_mlg/'+str(i+1)+'.txt')
    
    #perform rough feature normalisation 
    norm = np.mean(datgr[mini:maxi,1])
    
    high_frt.append(np.max(datfr[mini:maxi,1]/norm))
    low_frt.append(np.min(datfr[mini:maxi,1]/norm))  
    high_grt.append(np.max(datgr[mini:maxi,1]/norm))
    low_grt.append(np.min(datgr[mini:maxi,1]/norm))

    
total_low_test = np.concatenate((low_grt,low_frt), axis=None)
total_high_test = np.concatenate((high_grt,high_frt), axis=None)

# Create matrix of features 
test_x = np.concatenate(([total_low_test],[total_high_test]))
# Create vector of 'right' answers 
test_y = np.concatenate((np.zeros((test_set-1)),np.ones((test_set-1))))

# Calculate the accuracy 
myacc = 0

for i in range(test_set*2-2):
    if(predictor(test_x[0][i],test_x[1][i])==test_y[i]):
        myacc +=1 
        
print(myacc/(test_set*2-2))

In [ ]:
plt.plot(total_low, decbound(xopt,total_low))
plt.plot(low_frt,high_frt, linestyle='none',marker='x', label='f(R)')
plt.plot(low_grt,high_grt, linestyle='none',marker='o', label='GR')
plt.ylabel('Maximum $P(k)$ ',**font1)
plt.xlabel('Minimum $P(k)$', **font1)
plt.legend(loc='best',prop={'size': 16})